# Object-oriented scientific programming with C++

Matthias Möller, Jonas Thies, Cálin Georgescu, Jingya Li (Numerical Analysis, DIAM)

Lecture 2

# Task: Dot product

Write a C++ code that computes the dot product
$$
a \cdot b=\sum_{i=1}^n a_i b_i
$$
of two vectors $a=\left[a_1, a_2, \ldots, a_n\right]$ and $b=\left[b_1, b_2, \ldots, b_n\right]$ and terminates if the two vectors have different length.

# Dot product function

The main functionality of the ```dot_product``` function without any fail-safe checks

In [ ]:
double dot_product(const double* a, int n, const double* b, int m)
{
    double d=0.0;
    for (auto i=0; i<n; i++)
        d += a[i]*b[i];
    return d;
}

# Dot product function - improved version

First version of the ```dot_product``` function _with_ fail-safe checks

In [ ]:
#include <exception>

double dot_product(const double* a, int n, const double* b, int m)
{
    if (a == nullptr || b == nullptr) {
        // Handle null pointers by throwing an exception
        throw std::invalid_argument("Null pointer argument");
    }

    if (n != m) {
        // Handle mismatched sizes by throwing an exception
        throw std::invalid_argument("Array sizes mismatch");
    }

    // Core functionality
    double d = 0.0;
    for (int i = 0; i < n; ++i) {
        d += a[i] * b[i];
    }
    return d;
}

Creation of two vectors and use of the ```dot_product``` function

In [ ]:
#include <iostream>

double x[5] = { 1, 2, 3, 4, 5 };
double y[4] = { 1, 2, 3, 4 };

try {
    double d = dot_product(x, 5, y, 4);
} catch (const std::exception &e) {
    std::cout << e.what() << std::endl;
}

It would be much better if `x` and `y` would "know" their length internally so that the calling function cannot provide inconsistent data

# Cool! But what was the reason I enrolled in this course?
<center><img src='plots/flashback.png' width="400px"></center>

# Flashback: Object Oriented Programming

Imagine each LEGO block as a piece of data (like an integer, string, etc.). A ```struct``` or a ```class``` is like a LEGO set that contains a variety of different blocks.

- OOP: bundle data (e.g. ```array```) and functionality (e.g. ```length```) into a ```struct``` or ```class```   
- Components of a <span style = "color:red;">struct</span> are ```public``` (=can be accessed from outside the struct) by default
- Components of a <span style = "color:red;">class</span> are ```private``` (=cannot be accessed from outside the class) by default
- Components of a struct/class are <span style ="color:red;">attributes</span> and <span style = "color:red;">member
functions (=methods)</span>


# Class vs. struct

In [ ]:
class Vector {
    private: //default
    public:
        double* array;
        int length;
};

In [ ]:
struct Vector {
    public: // default
        double* array;
        int length;
    private:
};

__When to use ```class``` and when to use ```struct```?__

- ```struct``` is typically used when you want to group data together without needing to restrict access to it. It is straightforward and simple. Many type traits (later in this course) are implemented as ```struct```s.
- ```class``` is typically used when you want more control over the data and the interface through which it is accessed and manipulated, promoting the principles of encapsulation and data hiding.

# Dot product as a member function of Vector

Second version of the ```dot_product``` function

In [ ]:
#include <exception>

class Vector {
    public:
        double* array;
        int length;
    
    double dot_product(const Vector& a, const Vector& b)
    {
        if (a.length != b.length) 
            throw std::invalid_argument("Vector lengths mismatch");
        
        double d=0.0;
        for (auto i=0; i<a.length; i++)
            d += a.array[i]*b.array[i];
        return d;
    }    
};

Access of members of a ```struct``` or ```class``` by __dot-notation (".")__

Is the above implementation really OOP? How would you invoke it from the main program?

In [ ]:
#include <iostream>

Vector x,y;
x.array = new double[5]; x.length = 5;
y.array = new double[5]; y.length = 5;

for (int i = 0; i < 5; ++i) {
    x.array[i] = i + 1;  // 1, 2, 3, 4, 5
    y.array[i] = i + 1;  // 1, 2, 3, 4, 5
}
 
try {
    double result = /** ??? **/;
} catch (const std::exception &e) {
    std::cout << e.what() << std::endl;
}

The current implementation of the `dot_product` function comes from functional programming and is not OOP style because it takes two input arguments `x` and `y` and returns one output argument, the dot product. In other words, it is not attached to any object (`x` or `y`).

If we want to implement a function inside a ```class``` or ```struct``` that is not attached to an object we have to define it as ```static```

In [ ]:
#include <exception>

class Vector {
    public:
        double* array;
        int length;
    
    static double dot_product(const Vector& a, const Vector& b)
    {
        if (a.length != b.length) 
            throw std::invalid_argument("Vector lengths mismatch");
        
        double d=0.0;
        for (auto i=0; i<a.length; i++)
            d += a.array[i]*b.array[i];
        return d;
    }    
};

```static``` member functions are invoked with the full classname (comparable to namespaces)

In [ ]:
#include <iostream>

Vector x,y;
x.array = new double[5]; x.length = 5;
y.array = new double[5]; y.length = 5;

for (int i = 0; i < 5; ++i) {
    x.array[i] = i + 1;  // 1, 2, 3, 4, 5
    y.array[i] = i + 1;  // 1, 2, 3, 4, 5
}
 
try {
    double result = Vector::dot_product(x, y);
} catch (const std::exception &e) {
    std::cout << e.what() << std::endl;
}

- It is still possible to initialise ```x.length``` by the wrong value, e.g.,
   ```cpp
   x.array = new double[5] {1, 2, 3, 4, 5}; 
   x.length = 4;
   ```
- The main function is not very readable due to the lengthy declaration, initialization and deletion of data
- OOP solution:
  - __Constructor(s):__ <span style="color:red;">method</span> to construct a new <span style="color:red;">Vector object</span>
   - __Destructor:__ <span style="color:red;">method</span> to destruct an existing <span style="color:red;">Vector object</span>

# Constructor

The __constructor__ is called each time a new Vector object (=instance of the class Vector) is created

In [ ]:
class Vector
{
    public:
        double* array;
        int length;
    
    Vector() // Default constructor
    {
        array = nullptr;
        length = 0;
    }
};

A class can have __multiple constructors__ if they have a different interface (=different parameters)

In [ ]:
class Vector
{
    public:
        double* array;
        int length;
    
    Vector() // Default constructor
    {
        array = nullptr;
        length = 0;
    }
    
    Vector(int len) // Another constructor
    {
        array = new double[len];
        length = len;
    }
};

What if a parameter has the same name as an attribute?

In [ ]:
class Vector
{
    public:
        double* array;
        int length;
    
    Vector(int length) // Another constructor
    {
        // The 'this' pointer refers to the object itself,
        // hence this->length is the attribute and length
        // is the parameter passed to the constructor

        array = new double[length];
        this->length = length;
    }
}; 

# Destructor

The __destructor__ is called implicitly at the end of the lifetime of a Vector object, e.g., at the end of its scope

In [ ]:
class Vector
{
    public:
        double* array;
        int length;
    
    ~Vector() // Destructor (and there can be only one!)
    {
        delete[] array;
        length = 0;
    }
};

Cleaning up the main program with constructors and (implicitly invoked) destructors

```cpp
int main(){
    Vector x; // Default constructor is called
    {
        Vector y(5); // Constructor is called
        // Destructor is called for Vector y
    }
    // Destructor is called for Vector x
  }
```

Without ```array = nullptr``` in the default constructor the destruction of ```x``` will lead to a run-time error.

# Uniform initialisation constructors (C++11)

Remember this
```double x[5] = { 1, 2, 3, 4, 5 };```

It would be cool to simply write
```Vector x = { 1, 2, 3, 4, 5 };```


C++11 solution: __initializer lists__

```cpp
Vector(const std::initializer_list<double> list) {
    length = (int)list.size();
    array = new double[length]; 
    std::uninitialized_copy(list.begin(), list.end(), array);
}
```

In [ ]:
#include <initializer_list>
#include <memory>

class Vector {
private:
    double* array;
    int length;

public:
    // Constructor with initializer list
    Vector(const std::initializer_list<double>& list)
    {
        length = (int)list.size();
        array = new double[length];
        std::uninitialized_copy(list.begin(), list.end(), array);
    }
    
    // Destructor
    ~Vector() {
        delete[] array;
    }
};

# Dot product – close to perfection

Third version of the dot product using Vector class with __uniform initialisation constructor__ (C++11) and exceptions

```cpp
int main()
{
    Vector x = { 1, 2, 3, 4, 5 };
    Vector y = { 2, 4, 6, 8, 10};
    try {
        double dot_product(x, y);
        } catch (const std::exception &e) {
    std::cout << e.what() << std::endl;
    }
}
```

# Delegating constructor (C++11)

Can we delegate some of the work?

In [ ]:
#include <memory>
#include <initializer_list>

class Vector {
private:
    double* array;
    int length;

public:
    Vector(int length)
    {
        this->length = length;
        array = new double[length];
    }
    
    Vector(const std::initializer_list<double>& list)
    {
        length = (int)list.size();
        array = new double[length];
        std::uninitialized_copy(list.begin(), list.end(), array);
    } 
}    

__Delegating constructors__ delegate part of the work to
another constructor of the same or another class

```cpp
Vector(int length) 
    : length(length), array(new double[length])
{ }
```

- Here, delegation is not really helpful but more a question of coding style, e.g., some programmers use delegation in all situation where this is technically possible

- It is no longer necessary to distinguish between the __attribute__ (```this->length```) and the __argument__ (```length```) if both have the same name. But be careful with the order in which delegated objects are constructed!

# Quiz: Which of the following codes is wrong?

- Code 1
  ```cpp
  Vector(int len)
      : length(len), 
        array(new double[len])
    {}

  ```

- Code 2
  ```cpp
  Vector(int len)
      : array(new double[len]), 
        length(len)
    {}

  ```

- Code 3
  ```cpp
  Vector(int len)
      : length(len),
        array(new double[lengh])
    {}

  ```

- Code 4
  ```cpp
  Vector(int len)
    : array(new double[length]),
      length(len)
    {}

  ```

If you have multiple constructors with increasing functionality, __delegating constructors__ can be really helpful to remove duplicate code, e.g.

```cpp
Vector(const std::initializer_list<double>& list)
    : Vector((int)list.size())
    {
        std::uninitialized_copy(list.begin(), list.end(), array);
    }
```

# Turning a stand-alone function into a member function

Function that computes the sum of a Vector

```cpp
static double sum(const Vector& a)
{
    double s = 0;
    for (auto i=0; i<a.length; i++)
        s += a.array[i];
    return s;
}
```

This is not really OOP-style!

```cpp
int main() {
 Vector x = { 1, 2, 3, 4, 5 };
 std::cout << sum(x) << std::endl; 
}
```

# Implementation of ```sum``` as an OOP-style member function

In [ ]:
#include <memory>
#include <initializer_list>

class Vector {
    private:
        double* array;
        int length;
    
    public:
        Vector(const std::initializer_list<double>& list) {
            length = static_cast<int>(list.size());
            array = new double[length];
            std::uninitialized_copy(list.begin(), list.end(), array);
        }
        
        ~Vector() {
            delete[] array;
        }
    
        double sum() {
            double s = 0;
            for (int i = 0; i < length; i++) {
                s += array[i];
            }
            return s;
        }
};

 This is good OOP-style

```cpp
Vector v = {1, 2, 3};
std::cout << v.sum() << std::endl;
```

Can we implement the ```dot_product``` function as a member function?

In [ ]:
#include <exception>

class Vector {
    private:
        double* array;
        int length;
    
    public:
        double dot_product(const Vector& other) {
            if (length != other.length)
                throw std::invalid_argument("Vector lengths mismatch");
            
            double d=0.0;
            for (auto i=0; i<length; i++)
                d += array[i]*other.array[i];
            return d;
        }
};

This is good OOP-style

```cpp
int main()
{
    Vector x = {1,2,3}; 
    Vector y = {2,4,6};
    
    std::cout << x.dot_product(y) << std::endl;
    std::cout << y.dot_product(x) << std::endl;
}
```

Formally, the dot product is an operation between two
Vector objects and not a member function of one Vector
object that needs another Vector object for calculation

# Operator overloading

C++ allows to overload (=redefine) the standard operators
- Unary operators: ```++a```, ```a++```, ```--a```, ```a--```, ```~a```, ```!a```
- Binary operators: ```a+b```, ```a-b```, ```a*b```, ```a/b```
- Relational operators: ```a==b```, ```a!=b```, ```a<b```, ```a<=b```, ```a>b```, ```a>=b```

Interfaces:

```cpp
return_type operator()                    [const]
return_type operator(const Vector& other) [const]
```
 
[Complete list: https://en.cppreference.com/w/cpp/language/operators](https://en.cppreference.com/w/cpp/language/operators)

Implementation of dot product as __overloaded *-operator__

```cpp
double operator*(const Vector& other) const
    {
        if (length != other.length)
                throw std::invalid_argument("Vector lengths mismatch");
        double d=0.0;
        for (auto i=0; i<length; i++)
            d += array[i]*other.array[i];
        return d;
}
```

Now, the dot product is implemented as __*-operator__ that
maps two Vector objects to a scalar value

```cpp
int main()
{
    Vector x = {1,2,3}; 
    Vector y = {2,4,6};
    
    std::cout << x * y << std::endl;
    std::cout << y * x << std::endl;
}
```

The <span style=color:blue;>const</span> specifier indicates that the Vector reference ```other``` must not be modified by the __*-operator__

The trailing <span style=color:red;>const</span> specifier indicates that the ```this``` pointer (aka, the object whose function is invoked) must not be modified by the __*-operator__

<code>double operator*(<span style=color:blue;>const</span> Vector&amp other) <span style=color:red;>const</span> { ... }</code>

# Assignment by operator overloading

Implementation of assignment as __overloaded =-operator__

```cpp
Vector& operator=(const Vector& other)
{
    if (this != &other)
    {
        length = other.length;
        delete[] array;
        array = new double[length];
        for (auto i=0; i<length; ++i)
            array[i] = other.array[i];
    }
    return *this;
}
```

- Usage: 
  ```cpp
  Vector x, y = {1,2,3};
  x = y;
  ```
- Note that the ```this``` pointer is modified so there must not be a trailing ```const```

Implementation of incremental assignment as __overloaded =-operator__

```cpp
Vector& operator+=(const Vector& other)
{
    if(length != other.length) 
        throw std::invalid_argument("Vector lengths mismatch");
    for (auto i=0; i<length; i++)
        array[i] += other.array[i];
    return *this;
}
```

- Usage: 
  ```cpp
  Vector x = {1,2,3}, y = {4,5,6};
  x += y;
  ```
- Note that the ```this``` pointer is modified so there must not be a trailing ```const```

# Container class

In [ ]:
#include <iostream>
#include <initializer_list>
#include <algorithm>

class Container {
private:
    int size;
    double* elements;

public:
    // Default constructor
    Container() : size(0), elements(nullptr) {
        std::cout << "Default constructor called" << std::endl;
    }

    // Converting constructor: can also initialize Container with an int
    Container(int length) : size(length), elements(new double[length]) {
        std::cout << "Converting constructor (from int) called" << std::endl;
        std::fill(elements, elements + size, 0.0);
    }

    // Converting constructor: can also initialize Container with an initializer_list
    Container(const std::initializer_list<double>& list) 
    : size(static_cast<int>(list.size())), elements(new double[size]) {
        std::cout << "Converting constructor (from initializer_list) called" << std::endl;
        std::copy(list.begin(), list.end(), elements);
    }

    // Explicit constructor to prevent implicit conversions from a single float
    explicit Container(float initial_value) : size(1), elements(new double[1]) {
        std::cout << "Explicit constructor called" << std::endl;
        elements[0] = initial_value;
    }

    // Copy constructor
    Container(const Container& other) : size(other.size), elements(new double[other.size]) {
        std::cout << "Copy constructor called" << std::endl;
        std::copy(other.elements, other.elements + size, elements);
    }

    // Move constructor
    Container(Container&& other) noexcept : size(other.size), elements(other.elements) {
        std::cout << "Move constructor called" << std::endl;
        other.size = 0;
        other.elements = nullptr;
    }

    // Destructor
    ~Container() {
        std::cout << "Destructor called" << std::endl;
        delete[] elements;
    }

    // Copy assignment operator
    Container& operator=(const Container& other) {
        std::cout << "Copy assignment operator called" << std::endl;
        if (this != &other) { // Protect against self-assignment
            delete[] elements; // Free the existing resource.
            size = other.size;
            elements = new double[size];
            std::copy(other.elements, other.elements + size, elements);
        }
        return *this;
    }

    // Move assignment operator
    Container& operator=(Container&& other) noexcept {
        std::cout << "Move assignment operator called" << std::endl;
        if (this != &other) { // Prevent self-assignment
            delete[] elements; // Free the existing resource.
            size = other.size;
            elements = other.elements;
            other.size = 0;
            other.elements = nullptr;
        }
        return *this;
    }

    // A simple print function to show the contents
    void print() const {
        for (int i = 0; i < size; ++i) {
            std::cout << elements[i] << " ";
        }
        std::cout << std::endl;
    }
};

int main() {
    Container defaultContainer; // Calls default constructor

    Container fromInt = 5; // Calls converting constructor with an int
    fromInt.print();

    Container fromList = {1.0, 2.0, 3.0}; // Calls converting constructor with an initializer_list
    fromList.print();

    // The line below will cause a compile-time error due to the 'explicit' keyword
    // Container fromFloat = 2.5f; // Error
    Container fromFloat(2.5f); // Calls explicit constructor
    fromFloat.print();

    Container copyContainer = fromList; // Calls copy constructor
    copyContainer.print();

    // Assigning a new value to copyContainer using copy assignment operator
    copyContainer = fromInt;
    copyContainer.print();

    Container moveContainer; // Calls default constructor
    moveContainer = std::move(copyContainer); // Calls move assignment operator
    moveContainer.print();

    return 0;
}

# Container class

```cpp
class Container {
private:
    double* data;
    int length;
    
public:
    Container(int length)
    : length(length), data(new double[length])
    { }
    
    Container(const std::initializer_list<double> l)
    : Container( (int)l.size() )
    {
        std::uninitialized_copy(l.begin(), l.end(), data);
    }
};
```

# Conversion constructors

Both constructors convert a single input argument into a Container object, hence, they are called __conversion constructors__. They can be called in two different ways

- Using the regular construction form
  ```cpp
  Container a( 4 );
  Container a( {1,2,3,4} );
  ```
- Using copy initialisation
  ```cpp
  Container a = 4;         // -> Container a( 4 )
  Container a = {1,2,3,4}; // -> Container a( {1,2,3,4} )
  Container a = {4};       // which constructor is called?
  ```

# Explicit specifier

The __explicit specifier__ prevents the use of the constructor as conversion constructor

```cpp
explicit Container(int length) 
    : length(length), data(new double[length])
    { }
}
```

Now, copy-initialisation (```Container a = 4;```) is no longer possible
but explicit constructor (```Container a(  4 );```) has to be used

# Constructors summary

<table>
<center>
  <tr>
    <th style="text-align:left;">Constructor</th>
    <th style="text-align:left;">Description</th>
    <th style="text-align:left;">Usage</th>
  </tr>
  <tr>
    <td  style="text-align:left;">Default</td>
    <td  style="text-align:left;">Constructor with no parameters.</td>
    <td style="text-align:left;">Used to create an object with default values.</td>
  </tr>
  <tr>
    <td style="text-align:left;">Parameterized</td>
    <td style="text-align:left;">Constructor with parameters to initialize an object with specific values.</td>
    <td style="text-align:left;">Used to create an object with specified attributes.</td>
  </tr>
  <tr>
    <td style="text-align:left;">Copy</td>
    <td style="text-align:left;">A constructor that initializes an object using another object of the same class.</td>
    <td style="text-align:left;">Used to create a copy of an object.</td>
  </tr>
  <tr>
    <td style="text-align:left;">Explicit</td>
    <td style="text-align:left;">Constructor with the explicit keyword to prevent implicit conversions or copy-initialization.</td>
    <td style="text-align:left;">Used to enforce explicit object creation with constructor.</td>
  </tr>
</center>
</table>



# Task: Numerical integration

- Approximate a one-dimensional integral by numerical quadrature
  $$
  \int_a^bf(x)dx \approx \sum_{i=1}^n w_i f\left(x_i\right)
  $$
  
- Choice of quadrature weights $w_i$ and points $x_i$ determines the concrete numerical integration rule
    

# Simple integration rules

- Midpoint rule
  $$\int_a^b f(x) d x \approx(b-a) \cdot f\left(\frac{a+b}{2}\right)$$
- Simpson rule
  $$\int_a^b f(x) d x \approx \frac{b-a}{6}\left[f(a)+4 f\left(\frac{a+b}{2}\right)+f(b)\right]$$
- Rectangle rule
  $$\int_a^b f(x) d x \approx h \sum_{n=0}^{N-1} f\left(x_n\right), \quad h=\frac{b-a}{N}, \quad x_n=a+n h$$

# Gauss integration rules

- Zoo of Gauss integration rules with quadrature weights and points tabulated for the reference interval $[-1,1]$
- Complete list of weights/points is available, e.g., at Wikipedia

<table>
      <tr>
    <th style="text-align:left;">$n$</th>
    <th style="text-align:left;">$\xi_{i}$</th>
    <th style="text-align:left;">$w_i$</th>
  </tr>
      <tr>
    <td  style="text-align:left;">1</td>
    <td  style="text-align:left;">0</td>
    <td style="text-align:left;">2</td>
  </tr>
       <tr>
    <td  style="text-align:left;">2</td>
    <td  style="text-align:left;">-0.57735026919</td>
    <td style="text-align:left;">2</td>    
  </tr>
    <tr>
    <td  style="text-align:left;"></td>
    <td  style="text-align:left;">0.57735026919</td>
    <td style="text-align:left;">1</td>     
  </tr>
        <tr>
    <td  style="text-align:left;">3</td>
    <td  style="text-align:left;">-0.774596669241</td>
    <td style="text-align:left;">5/9</td>     
  </tr>
            <tr>
    <td  style="text-align:left;"></td>
    <td  style="text-align:left;">0.0</td>
    <td style="text-align:left;">8/9</td>     
  </tr>
            <tr>
    <td  style="text-align:left;"></td>
    <td  style="text-align:left;">0.774596669241</td>
    <td style="text-align:left;">5/9</td>     
  </tr>
            <tr>
    <td  style="text-align:left;">4</td>
    <td  style="text-align:left;">-0.861136311594053</td>
    <td style="text-align:left;">0.347854845137454</td>     
  </tr>
            <tr>
    <td  style="text-align:left;"></td>
    <td  style="text-align:left;">-0.339981043584856</td>
    <td style="text-align:left;">0.652145154862546</td>     
  </tr>
  <tr>
    <td  style="text-align:left;"></td>
    <td  style="text-align:left;">0.774596669241</td>
    <td style = "text-align:left;">0.652145154862546</td>
  </tr>
  <tr>
    <td  style="text-align:left;"></td>
    <td  style="text-align:left;">0.861136311594053</td>
    <td style ="text-align:left;">0.347854845137454</td>
  </tr>
    </table>

- Change of variable theorem
  $$\int_{a}^{b} f(x)dx = \int_{-1}^{1}f(\phi(t))\phi^{i}(t)dt$$
- Mapping from interval $[a,b]$ to interval $[-1,1]$
  $$\phi(t) = \frac{b-a}{2}t + \frac{a+b}{2}, \phi^{'}(t) = \frac{b-a}{2}$$
- Numerical quadrature rule
  $$\int_{a}^{b}f(x)dx\approx\phi^{'}\sum_{n=1}^{n}w_if(\phi(\xi_i))$$

# Program design

We need...
- A strategy to ensure that all numerical quadrature rules
(=classes) provide an __identical interface__ for evaluating integrals
- A standard way to __pass user-definable function__ f(x) from outside (=main routine) to the evaluation function

- <span style=color:gray;>A strategy to ensure that all numerical quadrature rules (=classes) provide an __identical interface__ for evaluating integrals</span>
  - __Polymorphism:__ <span style=color:blue;>Base class Quadrature</span> provides common attributes and member functions (at least their <span style=color:red;>interface declaration</span>); <span style=color:red;>derived classes implement</span> specific quadrature rule (reusing common functionality of the base class, where this is possible and makes sense)

- <span style=color:gray;>A standard way to __pass user-definable function__ f(x) from outside (=main routine) to the evaluation function</span>
  - Function pointers (traditional approach)
  - Lambda expressions (recommended approach since C++11)

# Function pointers

- Define a function to be integrated
  ```cpp
  const double myfunc1(double x){ return x; }
  ```
- Define interface of the integrate function
  ```cpp
  double integrate(const double (*func)(double x), double a, double b) { ... }
  ```

- Usage: 
  ```cpp
  integrate(myfunc1, 0, 1);
  ```

# Lambda expressions

- Introduced in C++11, __lambda expressions__ provide an elegant way to write user-defined callback functions
- General syntax
  ```cpp
  auto name = [<captures>] (<parameters>) { <body> };
  ```
- Lambda expressions can be inlined (anonymous functions)
  ```cpp
  integrate([<captures>] (<parameters>) { <body> });
  ```

- Define function to be integrated
  ```cpp
  auto myfunc2 = [](double x) { return x; };
  ```
- Define interface of the integration function
  ```cpp
  double integrate(std::function<double(double)> func, double a, double b) const { ... }
  ```
- Usage:
  ```cpp
  integrate(myfunc2, 0, 1);
  integrate([](double x){ return x; }, 0, 1);
  ```

# Program design, revisited

<center><img src='plots/lecture2_program_design.png' width="700px"></center>

# Base class ```Quadrature```

In [ ]:
class Quadrature
{
    public:
        Quadrature()
        : n(0), weights(nullptr), points(nullptr) {};
    
        Quadrature(int n)
        : n(n), weights(new double[n]), points(new double[n]) {};
    
        ~Quadrature()
        { delete[] weights; delete[] points; n=0; }
    
    private:
        double* weights;
        double* points;
        int     n;
};

_Scenario I:_ We want to __declare the interface__ of the integrate function but we want to _force_ the user to implement each integration rule individually

```cpp
// pure (=0) virtual member functions
virtual double integrate(double (*func)(double x), double a, double b) const = 0;
virtual double integrate(std::function<double(double)> func, double a, double b) const  = 0;
```

- Keyword ```virtual ... = 0;``` declares the function to be __pure virtual__
- That is, each class that is derived from the __abstract class__ ```Quadrature``` must(!!!) implement this function explicitly
- Otherwise, the compiler complains when the programmer forgets to implement a pure virtual function and tries to create an object of the derived but not fully implemented class

# Abstract classes

A class with at least one pure virtual function is an __abstract class__ and it is not possible to create an object thereof

<img src='plots/lecture2_abstract_classes.png' width="400px">

# Base class ```Quadrature```

_Scenario II:_ We provide a _generic implementation_ but allow the user to override it explicitly in a derived class

```cpp
// virtual memper functions
virtual double integrate(double (*func)(double x), double a, double b) const {...}
virtual double integrate(std::function<double(double)> func, double a, double b) const {...}
```
- Keyword ```virtual``` declares the function __virtual__. Virtual functions _can_ be overridden in a derived class. If no overriding takes place, then the function implementation from the base class is used

In [ ]:
class Quadrature {
    private:
        double* weights;
        double* points;
        int     n;
    
    public:
        Quadrature()
        : n(0), weights(nullptr), points(nullptr) {};
    
        Quadrature(int n)
        : n(n), weights(new double[n]), points(new double[n]) {};
    
        ~Quadrature()
        { delete[] weights; delete[] points; n=0; }
    
    // pure virtual functions (must be implemented in derived class)
    virtual double mapping(double xi, double a, double b) const = 0;
    virtual double factor(double a, double b) const = 0;
    
    // virtual function (generic implementation)
    virtual double integrate(double (*func)(double x), double a, double b) const {
        double integral(0.0);
        for (auto i=0; i<n; i++)
            integral += weights[i]*func(mapping(points[i],a,b)); 
        return factor(a,b)*integral;
    } 
};

- The __virtual__ ```integrate``` function makes use of the __pure virtual__ functions ```factor``` and ```mapping```
- Both functions are __not__ implemented in class ```Quadrature```
- It is therefore obvious that class ```Quadrature``` must be an __abstract class__ (and cannot be instantiated) since some of its functions (here: ```integrate```) are still unavailable
- Virtual functions make it is possible to call functions in the base class which will be implemented in the derived class

# Class ```MidpointRule```

- __Derive__ class ```MidpointRule``` from base class ```Quadrature```

In [ ]:
class MidpointRule : public Quadrature
{
    // Implement pure virtual functions (not used but need to be implemented!)
    virtual double mapping(double xi, double a, double b) const  { return 0; }
    virtual double factor(double a, double b) const { return 1; }
    
    // Override the implementation of the virtual integrate 
    // function from class Quadrature with own implementation 
    virtual double integrate(double (*func)(double x), double a, double b) const
    {
        return (b-a) * func( 0.5 * (a+b) );
    }
};

# Class ```SimpsonRule```

- __Derive__ class ```SimpsonRule``` from base class ```Quadrature```

In [ ]:
class SimpsonRule : public Quadrature
{
    // Implement pure virtual functions (not used but need to be implemented!)
    virtual double mapping(double xi, double a, double b) const  { return 0; }
    virtual double factor(double a, double b) const { return 1; }
    
    // Override the implementation of the virtual integrate 
    // function from class Quadrature with own implementation 
    virtual double integrate(double (*func)(double x), double a, double b) const
    {
        return (b-a)/6.0 * ( func(a) + 4.0 * func( 0.5*(a+b) ) + func(b) );
    }
};

# Class ```GaussRule```

__Derive__ class ```GaussRule``` from base class ```Quadrature```

```cpp
class GaussRule : publixc Quadrature {
    GaussRule(int n) : Quadrature(n) {
        switch (n) {
            case 1: 
                weights[0] = { 2.0 }; // QUIZ: Do we have access to the
                points[0]  = { 0.0 }; // attributes weights and points ?
                break;
            case 2:
                ...
            default:
                throw "Invalid argument";
        }
    }
};
```

# Program design, revisited

<center><img src='plots/lecture2_program_design_02.png' width="700px"></center>

# Program design, revisited

<center><img src='plots/lecture2_program_design_03.png' width="700px"></center>

# Class ```GaussRule```

- Attributes from base class are now visible in derived class
- Class ```GaussRule``` implements functions ```factor``` and ```mapping```
- Class ```GaussRule``` inherits the virtual function ```integrate``` from class ```Quadrature```

In [ ]:
class GaussRule : public Quadrature
{
    virtual double factor(double a, double b) const 
    { return 0.5 * (b-a); }
    
    virtual double mapping(double xi, double a, double b) const 
    { return 0.5 * (b-a) * xi + 0.5 * (a+b); }
};

# Keyword: ```override``` (C++11)

- With the ```override``` keyword you can force the compiler to explicitly check that the function in a derived class __overrides__ a (pure) virtual function from the base class

In [ ]:
class GaussRule : public Quadrature
{
    virtual double factor(double a, double b) const override
    { 
        return 0.5 * (b - a); 
    }
};

- If the base class Quadrature does not specify a (pure) virtual function ```factor``` an error will be thrown

# Keyword: ```final``` (C++11)

- With the ```final``` keyword you can force the compiler to explicitly prevent further overriding of functions

In [ ]:
class GaussRule : public Quadrature
{
    virtual double factor(double a, double b) const final
    { return 0.5*(b-a); }
};

- If a class ```GaussRuleImproved``` derived from ```GaussRule``` tries to override the function ```factor``` an error will be thrown

In [ ]:
class GaussRuleImproved : public GaussRule
{
    virtual double factor(double a, double b) const override
    { return 0.5*(b-a); }
};